In [6]:
#Sri Rama Jayam
import os
from dotenv import load_dotenv
import requests
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
from rank_bm25 import BM25Okapi
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.tools import Tool
from langchain.prompts import ChatPromptTemplate
import time
import numpy as np

# Set API Key in Environment
os.environ["OPENAI_API_KEY"] = "the apikey"
# Web Search using DuckDuckGo
def web_search(query, num_results=3):
    """Fetches search results from DuckDuckGo."""
    with DDGS() as ddgs:
        results = list(ddgs.text(query, max_results=num_results))
        if not results:
            print("[ERROR] No search results found.")
            return []
    return results

# Scrape Web Page Content
def page_scrape(url):
    """Fetches and scrapes webpage content."""
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                          "(KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, "html.parser")
        paras = [p.get_text() for p in soup.find_all("p")]
        return "\n".join(paras).strip()
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return ""

# Load Documents from Search
def load_documents(query, num_results=3):
    """Fetches search results and scrapes webpage content."""
    search_results = web_search(query, num_results)
    documents = []

    for result in search_results:
        url = result["href"]
        print(f"Fetching {url}")
        content = page_scrape(url)

        if content:
            documents.append({
                "url": url,
                "title": result["title"],
                "content": content
            })
        time.sleep(1)

    return documents

# BM25 Retriever
class BM25Retriever:
    def __init__(self, documents):
        self.documents = documents
        self.texts = [doc["content"] for doc in documents]
        self.tokenized_texts = [text.split() for text in self.texts]
        self.bm25 = BM25Okapi(self.tokenized_texts)

    def retrieve(self, query, k=3):
        query_tokens = query.split()
        scores = self.bm25.get_scores(query_tokens)
        return scores, self.documents

# Hybrid Retriever combining BM25 and FAISS
class HybridRetriever:
    def __init__(self, documents, bm25_weight=0.5):
        self.documents = documents
        self.bm25_retriever = BM25Retriever(documents)
        self.embeddings = OpenAIEmbeddings()
        texts = [doc["content"] for doc in documents]
        if texts:
            self.vectorstore = FAISS.from_texts(texts, self.embeddings)
        else:
            self.vectorstore = None
        self.bm25_weight = bm25_weight

    def retrieve(self, query, k=3):
        if not self.documents:
            return []

        # Get BM25 scores
        bm25_scores, docs = self.bm25_retriever.retrieve(query)

        # Get vector similarity scores
        if self.vectorstore:
            vector_results = self.vectorstore.similarity_search_with_score(query, k=len(self.documents))
            vector_scores = [1.0 - score for _, score in vector_results]  # Convert distance to similarity
        else:
            vector_scores = [0] * len(self.documents)

        # Normalize scores
        if max(bm25_scores) > 0:
            bm25_scores = [score / max(bm25_scores) for score in bm25_scores]
        if max(vector_scores) > 0:
            vector_scores = [score / max(vector_scores) for score in vector_scores]

        # Combine scores
        combined_scores = [self.bm25_weight * bm25 + (1 - self.bm25_weight) * vector
                           for bm25, vector in zip(bm25_scores, vector_scores)]

        # Get top k documents
        top_indices = sorted(range(len(combined_scores)), key=lambda i: combined_scores[i], reverse=True)[:k]
        return [self.documents[i] for i in top_indices]

# Retrieve Documents using Hybrid Search
def search_docs(query, hybrid_retriever):
    """Fetches relevant documents using hybrid search."""
    top_docs = hybrid_retriever.retrieve(query)
    if not top_docs:
        return "No relevant information found."
    return "\n\n".join([f"[Source: {doc['url']}]\n{doc['content'][:500]}..." for doc in top_docs])

# Main execution
if __name__ == "__main__":
    query = input("Enter your question: ")
    print("Loading documents...")
    documents = load_documents(query)

    # Build hybrid retriever
    hybrid_retriever = HybridRetriever(documents, bm25_weight=0.6)  # Adjust weight as needed

    # Define Search Tool
    tools = [
        Tool(
            name="WebContentSearch",
            func=lambda q: search_docs(q, hybrid_retriever),
            description="Searches for information in the web content database using hybrid search."
        )
    ]

    # Initialize LLM
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    # Define Prompt for LLM Agent
    system_prompt = """You are an AI assistant that finds information from web sources.
    - Use the WebContentSearch tool to find relevant information.
    - If no relevant information is found, return 'No relevant information available.'
    - Do not call WebContentSearch more than once per query.
    - Do not repeat queries or enter a loop."""

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{input}"),
        ("ai", "{agent_scratchpad}")
    ])

    # Create LLM Agent
    agent = create_openai_tools_agent(llm, tools, prompt)
    agent_executor = AgentExecutor(
        agent=agent,
        tools=tools,
        verbose=True,
        handle_parsing_errors=True,
        max_iterations=10
    )

    # Run the agent
    response = agent_executor.invoke({"input": query, "agent_scratchpad": ""})
    print("\nFINAL ANSWER:")
    print(response["output"])


Enter your question: Difference between saving and investment
Loading documents...
Fetching https://keydifferences.com/difference-between-savings-and-investment.html
Fetching https://www.usbank.com/financialiq/invest-your-money/investment-strategies/saving-vs-investing-whats-the-difference.html
Fetching https://www.bankrate.com/investing/saving-vs-investing/


> Entering new AgentExecutor chain...

Invoking: `WebContentSearch` with `difference between saving and investment`


[Source: https://www.bankrate.com/investing/saving-vs-investing/]
Compare accounts
Get guidance
Compare accounts
Get guidance
Money market accounts
Money market accounts are similar to savings accounts, but offer some checking features as well.
Get guidance
Banking
Unlock financial rewards by signing up for a savings or checking account with a bonus offer.
Get guidance
Compare rates
Get guidance
Compare rates
Get guidance
Buying & selling
Find an expert who knows the market. Compare trusted real estate agents all 

In [1]:
!pip install langchain langchain_community langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.49
    Uninstalling langchain-core-0.3.49:
      Successfully uninstalled langchain-core-0.3.49
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.7
    Uninstalling langchain-text-splitters-0.3.7:
      Successfully uninstalled langchain-text-splitters-0.3.7
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.22
    Uninstalling langchain-0.3.22:
  

In [2]:
!pip install rank_bm25 duckduckgo_search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 41.5 MB/s eta 0:00:00


In [5]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 33.7 MB/s eta 0:00:00
